# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

cities_df = pd.read_csv('../output_data/cities_t.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ibrā’,22.6906,58.5334,78.57,37,100,2.80,OM,1618714021
1,Ushuaia,-54.8000,-68.3000,51.80,58,40,11.50,AR,1618713586
2,Nome,64.5011,-165.4064,33.80,75,1,6.91,US,1618713961
3,Seoul,37.5683,126.9778,55.40,44,20,11.50,KR,1618713954
4,Nanortalik,60.1432,-45.2371,25.16,53,0,25.41,GL,1618714021
...,...,...,...,...,...,...,...,...,...
549,Terrace Bay,48.7834,-87.1000,30.20,86,15,2.30,CA,1618714206
550,Porbandar,21.6422,69.6093,77.00,100,19,5.91,IN,1618714206
551,Ayan,56.4500,138.1667,28.04,89,98,4.27,RU,1618714207
552,Maragogi,-9.0122,-35.2225,76.21,83,92,2.68,BR,1618714207


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[['Lat', 'Lng']].astype(float)
humidity = cities_df['Humidity'].astype(float)

In [4]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_temp_df = cities_df[(cities_df['Max Temp'] >= 70) & (cities_df['Max Temp'] <= 80)]
cities_temp_ws_df = cities_temp_df[(cities_temp_df['Wind Speed'] < 10) & (cities_temp_df['Cloudiness'] == 0)]
# cities_temp_df.to_csv('../output_data/cities_maxtemp.csv', index=False)
cities_temp_ws_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Aquidauana,-20.4711,-55.7872,73.04,75,0,3.49,BR,1618714024
33,Hermanus,-34.4187,19.2345,73.00,21,0,3.00,ZA,1618714030
62,Busselton,-33.6500,115.3333,71.01,57,0,3.00,AU,1618714039
96,Conde,-7.2597,-34.9075,73.40,100,0,5.75,BR,1618714050
105,Adrar,20.5022,-10.0711,70.12,24,0,9.60,MR,1618714053
119,Cape Town,-33.9258,18.4232,73.00,71,0,2.30,ZA,1618713952
183,Pitimbu,-7.4706,-34.8086,73.40,100,0,5.75,BR,1618714080
263,Doha,25.2867,51.5333,78.80,25,0,9.22,QA,1618714106
330,Belmonte,-15.8631,-38.8828,71.60,88,0,4.54,BR,1618714129
340,Safaga,26.7292,33.9365,78.80,13,0,8.05,EG,1618714132


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities_temp_ws_df.reset_index()
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,14,Aquidauana,-20.4711,-55.7872,73.04,75,0,3.49,BR,1618714024
1,33,Hermanus,-34.4187,19.2345,73.00,21,0,3.00,ZA,1618714030
2,62,Busselton,-33.6500,115.3333,71.01,57,0,3.00,AU,1618714039
3,96,Conde,-7.2597,-34.9075,73.40,100,0,5.75,BR,1618714050
4,105,Adrar,20.5022,-10.0711,70.12,24,0,9.60,MR,1618714053
5,119,Cape Town,-33.9258,18.4232,73.00,71,0,2.30,ZA,1618713952
6,183,Pitimbu,-7.4706,-34.8086,73.40,100,0,5.75,BR,1618714080
7,263,Doha,25.2867,51.5333,78.80,25,0,9.22,QA,1618714106
8,330,Belmonte,-15.8631,-38.8828,71.60,88,0,4.54,BR,1618714129
9,340,Safaga,26.7292,33.9365,78.80,13,0,8.05,EG,1618714132


In [8]:
hotel_df['Hotel Name'] = ''
# params dictionary to update each iteration
params = {
    "location": '25.0213, 37.2685',
    "radius": 100000,
    "types": "Hotel",
    "keyword": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
hotel = requests.get(base_url, params=params).json()
hotel["results"][0]["name"]




'Royal Tours Permanent Camp'

In [9]:
hotel_df['Hotel Name'] = ''
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "Hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

        # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params).json()
   
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel["results"][0]["name"]
    except:
        print(f'Missing cities {lat}, {lng}')
hotel_df

Missing cities -7.2597, -34.9075
Missing cities 20.5022, -10.0711
Missing cities 22.6783, -3.9836


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,14,Aquidauana,-20.4711,-55.7872,73.04,75,0,3.49,BR,1618714024,Portal Pantaneiro Hotel
1,33,Hermanus,-34.4187,19.2345,73.00,21,0,3.00,ZA,1618714030,Abalone Lodge
2,62,Busselton,-33.6500,115.3333,71.01,57,0,3.00,AU,1618714039,The Sebel Busselton
3,96,Conde,-7.2597,-34.9075,73.40,100,0,5.75,BR,1618714050,
4,105,Adrar,20.5022,-10.0711,70.12,24,0,9.60,MR,1618714053,
5,119,Cape Town,-33.9258,18.4232,73.00,71,0,2.30,ZA,1618713952,Atlanticview Cape Town Boutique Hotel
6,183,Pitimbu,-7.4706,-34.8086,73.40,100,0,5.75,BR,1618714080,Asenza Beach Resort
7,263,Doha,25.2867,51.5333,78.80,25,0,9.22,QA,1618714106,The Curve Hotel
8,330,Belmonte,-15.8631,-38.8828,71.60,88,0,4.54,BR,1618714129,Pousada do Diogo
9,340,Safaga,26.7292,33.9365,78.80,13,0,8.05,EG,1618714132,Lamar Resort Abu Soma


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
locations1 = cities_df[['Lat', 'Lng']].astype(float)
humidity1 = cities_df['Humidity'].astype(float)
# hotel1 = hotel_df('Hotel Name').astype(str)
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
    }
fig = gmaps.figure(figure_layout)

heat_layer = gmaps.heatmap_layer(locations1, weights=humidity1)
marker = gmaps.marker_layer(locations, info_box_content=[f"Poverty Rate: {hotel}" for hotel in hotel_info])

fig.add_layer(marker)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))